In [61]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import time

#chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

# try:
#     driver = webdriver.Chrome(f'91/chromedriver')   
# except:
#     chromedriver_autoinstaller.install(True)
#     driver = webdriver.Chrome(f'91/chromedriver')

driver = webdriver.Chrome(f'../chromedriver')



In [63]:
driver.get("https://v4.map.naver.com")
driver.find_elements_by_css_selector("button.btn_close")[1].click()
# spmc_zoomout
driver.implicitly_wait(10)
#         a:nth-child(3)#

search_box = driver.find_element_by_css_selector("input#search-input")
search_box.send_keys("여수")

search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)
search_box.clear()

#지도레벨 축소
# for i in range(3):    
#     driver.find_elements_by_css_selector("a.spmc_zoomout")[0].click()
#     time.sleep(0.25)


#print(chrome_ver)

## store ids 
store_ids = [];

In [53]:
store_id_path = 'store_id.txt'

def get_store_id(webl_obj) :
    for store in webl_obj:
        str_id = store.get_attribute("class") + ""
        store_ids.append(str_id.split(',')[1]);
        print(str_id.split(',')[1])
        WriteTxtFiles(store_id_path, str_id.split(',')[1])


def WriteTxtFiles(path, store_id):    
    with open(path, "a") as file:
        file.write(store_id + ",")
        file.close()



In [64]:
search_query  = ['식당']
search_box = driver.find_element_by_css_selector("input#search-input")
search_box.clear()
current_page_num = 1;
max_search_page_num = 40;

search_box.send_keys(search_query)
#         a:nth-child(3)#
search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)

# 우선 store_id는 100개만 수집
for i in range(max_search_page_num):
    next_button = driver.find_element_by_css_selector("div.paginate").find_elements_by_tag_name("a")    
    stores = driver.find_elements_by_css_selector("ul.lst_site>li")
    get_store_id(stores)
    current_page_num += 1;
    
    for nextItem in next_button:
        if(nextItem.text == "다음"):
            nextItem.click()
            time.sleep(1)
            break
            
        if(nextItem.text == str(current_page_num)):
            nextItem.click()
            time.sleep(1)
            break
 

1758826380
34178356
37187006
11724047
13339318
32821525
13117981
17206525
283404739
1739065848
17397336
1180462194
11640716
259988078
1938552303
1196736265
20553144
12782652
17397391
36368164
1133190983
13174772
33175422
17397315
1754154655
12766975
1329260848
17206639
1611707408
36932623
11771679
36469012
1762882836
10976443
1103557752
17323433
1485842974
1445941496
111772984
1556751006
1471358179
1875939362
1567789211
770520048
21076773
16864591
1137078892
1317246570
11717077
1921762774
19568733
1006319468
20061767
19564206
17204716
20012131
1639879599
11866802
1079183154
16857822
12456350
1346534695
37331975
33292773
16931408
37116179
17010236
1699889912
1506039374
1647293481
1092218272
17292266
37698989
34905407
32231731
12020703
13488622
1159567937
13126734
1329944890
1431285703
1118669862
34346710
32231733
13521885
1901648708
16890925
34935020
16864099
1763714245
1787911441
1251395571
11814567
1057029065
1942925786
17397437
1506939181
17206783
1739359594
1271189319
1674261845
185

In [65]:
# 수집된 store_id로 방문자 리뷰 데이터를 수집한다
from bs4 import BeautifulSoup
import requests

with open("store_id.txt", "r") as file:
    store_ids = file.read()
    file.close()
    
driver = webdriver.Chrome(f'../chromedriver')

for store_id in store_ids.split(","):
    if(store_id):
        print("================")
        
        url = f'https://pcmap.place.naver.com/restaurant/{store_id}/review/visitor'
        htm = driver.get(url)
        time.sleep(1)
        
        try:
            for i in range(50):
                more_btn = driver.find_element_by_class_name("_3iTUo")
                if(more_btn):
                    more_btn.click()
                    time.sleep(0.75)
                else:
                    break;
        except Exception as ex:
            print("error catch : " , ex)
        
        print("id : " + store_id)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # li _2Cv-r -> 부모 컨테이너 // span _2tObC  ->별점 // span WoYOw ->리뷰 내용

        for ll_item in soup.find_all('li',{'class' : '_2Cv-r'}):

            rating_ll = ll_item.find('span',{'class' : '_2tObC'})
            review_ll = ll_item.find('span',{'class' : 'WoYOw'})

            if(review_ll):
                #10자 이상 리뷰만 의미있는 값으로 본다.
                if(len(review_ll.text)>10):
                    #3점 ~ 4점대 리뷰는 거른다 (3점미만 : 부정 , 5점 : 긍정)
                    if(rating_ll.text < 3 || rating_ll.text > 4.5)
                        review_content = review_ll.text
                        label = "0" #0 : 긍정, 1 부정
                        if("\n" in review_ll.text):
                            review_content = review_ll.text.replace("\n"," ")
                            
                        #3점 미만이면 부정평가
                        if(float(rating_ll.text) < 3 ):
                            label = "1";
                            
                        with open("review_data.txt", "a") as file:
                            # id , 별점, 리뷰내용, 레이블(0:긍정, 1:부정)
                            file.write(store_id+"\t" + rating_ll.text+"\t" + review_content +"\t"+ label+ "\n") 
    #                     print("rating : " + rating_ll.text)
    #                     print("review : " + review_ll.text)
        file.close()
#        driver.close()
        

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1482646633
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1015730987
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 36453756
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1615427494
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1947007050
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 13117981
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 17206525
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 283404739
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1739065848
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 17397336
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id 

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1079183154
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 16857822
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 12456350
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1346534695
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 37331975
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 35489024
id : 1574532392
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 34646938
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1329754486
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 21568910
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 36651302
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 13544217
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 30811307
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 35482294
error catch :  Message: stale element reference: element is not attached to the page document
  (Session info: chrome=91.0.4472.101)

id : 197836919
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1924307333
id : 36654606
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1104

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1766570556
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1367591955
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 37869736
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1208267266
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 32217691
error catch :  Message: stale element reference: element is not attached to the page document
  (Session info: chrome=91.0.4472.101)

id : 12830067
error 

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 16863852
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 38510173
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1033400481
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1487933958
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1824649062
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)



error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1969166053
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1099847943
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1590968639
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1413682526
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 16963433
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1108011503
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 38498113
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 20117113


KeyboardInterrupt: 